In [1]:
import numpy as np
import matplotlib.pyplot as plt
import GPy
%matplotlib inline
np.set_printoptions(suppress=True, precision=10)
from paramz.transformations import Logexp
import itertools

def monte_carlo_int(m,steps=1500):
    """
    My quick and dirty monte carlo integration.
    Returns a mean and the bounds of the 95%
    confidence interval (and the number of parameters
    it is integrating over).
    """
    m.optimize()
    opt = m.optimizer_array[None,:].copy()
    nparams = len(m.optimizer_array)
    searchwidth = 8
    V = searchwidth**nparams
    randnos = (np.random.rand(steps,nparams)-0.5)*searchwidth
    #randnos[:,2]=0
    test_params = randnos+opt.repeat(steps,0)[:]
    tot = []
    for i in range(steps):
        tot.append(np.exp(-m._objective(test_params[i,:])))
    mean = V*np.mean(tot)
    ste = V*np.std(tot)/np.sqrt(len(tot))
    return mean,mean-ste*1.96,mean+ste*1.96,nparams

warning in stationary: failed to import cython module: falling back to numpy


Here we look at $p(y|M_i, X)$ for two models ($i=1, i=2$). In each we integrate over all the hyperparemeters. The difference is how wide the Normal prior is over the lengthscale. I decided to use this as a way of modifying the model complexity as fixing parameters etc seems a bit less well defined.

We can see a tiny increase in the log likelihood, from -107.4 to -106.7, this was barely perceptible to be honest, I was sort of expecting a bigger increase in the marginal likelihood (over hyperparameters).

(note that as X and Y will change, you'll need to insert a new number into the prior if you want to run this again).

In [5]:
X = np.arange(0,40,1)[:,None]
Y = 10*(np.sin(X/5)+np.random.randn(X.shape[0],X.shape[1])*0.3)

In [4]:
for fixed in [False,True]:
    k = GPy.kern.RBF(1)
    #create model and optimise
    m2 = GPy.models.GPRegression(X,Y,k)
    if fixed:
        m2.kern.lengthscale.unconstrain()
        m2.kern.lengthscale.set_prior(GPy.priors.Gaussian(8.47,0.06))
    else:
        m2.kern.lengthscale.unconstrain()
        m2.kern.lengthscale.set_prior(GPy.priors.Gaussian(8.47,2.5))
    m2.optimize()
    
    if fixed:
        print "==lengthscale fixed==="
    else:
        print "===lengthscale unfixed==="
    print m2
    best = m2.param_array[:].copy()    
    m2.optimize()
    print np.log(monte_carlo_int(m2,30000))[:3]

===lengthscale unfixed===

Name : GP regression
Objective : 112.223201473
Number of Parameters : 3
Number of Optimization Parameters : 3
Updates : True
Parameters:
  GP_regression.           |          value  |  constraints  |    priors   
  rbf.variance             |  95.4499726575  |      +ve      |             
  rbf.lengthscale          |  8.58564552841  |               |  N(8.5, 2.5)
  Gaussian_noise.variance  |  8.51925249128  |      +ve      |             
[-107.4569539934 -107.4701229028 -107.4439562525]
==lengthscale fixed===

Name : GP regression
Objective : 108.496101584
Number of Parameters : 3
Number of Optimization Parameters : 3
Updates : True
Parameters:
  GP_regression.           |          value  |  constraints  |     priors   
  rbf.variance             |  92.2117800071  |      +ve      |              
  rbf.lengthscale          |  8.47015995706  |               |  N(8.5, 0.06)
  Gaussian_noise.variance  |  8.52882305365  |      +ve      |              
[-106.7086604

## Summary

My main concern is now that comparing the different models (with either the data combined or independently fitting, with the offset parameter in the combined model etc) might not really be very robust using this method.

The LOO x-validation method largely will incorporate model complexity automatically (if you're over fitting, then the left-out test data point will be poorly fitted). Also I need not worry about trying to integrate something that might be quite tricky/funny shape.

The only downside is the problem that Alan pointed out - I'm not really accounting for the full uncertainty as I'm just using the MAP estimate. After spending far too much time on this I think that this is probably an acceptable compromise!